# Imports

In [ ]:
import pandas as pd
import numpy as np
from nn_spikes import NeuralNetwork, batchTrain
from spike_tools import classifySpikesMLP, getSpikeWaveforms
import plotly.express as px
from simulated_annealing import anneal

In [ ]:
data = pd.read_csv('./datasources/spikes/training_data.csv')
spikeLocations = pd.read_csv('./datasources/spikes/training_spike_locations.csv', index_col=0)

data_training = pd.read_csv('./datasources/spikes/dev/data_training_SA.csv')
data_training.set_index(data_training.columns[0], drop=True, inplace=True)
data_training.index.name='index'
data_training.head(3)

data_validation = pd.read_csv('./datasources/spikes/dev/data_validation_SA.csv')
data_validation.set_index(data_validation.columns[0], drop=True, inplace=True)
data_validation.index.name='index'
data_validation.head(3)

spikeIndexes_training = pd.read_csv('./datasources/spikes/dev/spikeIndexes_training_SA.csv')
spikeIndexes_training.set_index(spikeIndexes_training.columns[0], drop=True, inplace=True)
spikeIndexes_training = spikeIndexes_training.values.flatten()
spikeIndexes_training[:5]

spikeIndexes_validation = pd.read_csv('./datasources/spikes/dev/spikeIndexes_validation_SA.csv')
spikeIndexes_validation.set_index(spikeIndexes_validation.columns[0], drop=True, inplace=True)
spikeIndexes_validation = spikeIndexes_validation.values.flatten()
spikeIndexes_validation[:5]

# Run simulated annealing optimiser

In [ ]:
# epochs, hidden_nodes, lr
solution = [150,1000,0.3] 

# Simulated annealing optimisation
results = anneal(solution, spikeLocations, iterations=4, alpha=0.6, variation=0.3)

In [ ]:
results

In [ ]:
results['windowSizes'] = [a[0] for a in results['Solution']]
results['hiddenNodes'] = [a[1] for a in results['Solution']]
results['initialLR'] = [a[2] for a in results['Solution']]

In [ ]:
results

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=results.index, y=results['windowSizes'], name="Window Size"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=results.index, y=results['hiddenNodes'], name="Hidden Nodes"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=results.index, y=results['initialLR'], name="Initial LR"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=results.index, y=results['Temperature'], name="Temperature"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=results.index, y=results['Error'], name="Error"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Window and InitialLR"
)

# Set x-axis title
fig.update_xaxes(title_text="combination")

# Set y-axes titles
fig.update_yaxes(title_text="Model Parameters", secondary_y=False)
fig.update_yaxes(title_text="Error", secondary_y=True)

fig.show()

In [ ]:
pb = str(round(99.9-results.loc[20, 'Error'], 2))
print("Best performance", pb+'%')

In [ ]:
fpath = './datasources/spikes/dev/sa_results_seed_' + str(results.loc[0,'windowSizes']) + '_' + str(results.loc[0,'hiddenNodes']) + '_' + str(results.loc[0,'initialLR']) + '_' + pb + '.csv'
print(fpath)

In [ ]:
results.to_csv(fpath)